In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [13]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [14]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.loc[:,"market_price_lag96h"] = df_bbidding1["market_price"].shift(192)
df_bbidding1.loc[:,"imbalance_price_lag96h"] = df_bbidding1["imbalance_price"].shift(192)
df_bbidding1.loc[:,"day_ahead_price_lag1week"] = df_bbidding1["day_ahead_price"].shift(336)
df_bbidding1.loc[:,"volume_lag96h"] = df_bbidding1["volume"].shift(192)
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2021-03-04 08:00:00+00:00,68.55,73.77,1201.25,17,-0.500000,0.467359,106.202828,126.692556,188.827864,...,248.421344,278.463233,323.809394,373.852140,463.068992,89.95,7.54,8.00,32.38,755.80
337,2021-03-04 08:30:00+00:00,69.25,73.77,1105.35,18,-0.500000,0.467359,109.169687,146.438753,203.691171,...,237.873544,290.936786,336.975817,382.262617,516.284064,89.00,-9.71,7.00,32.38,900.35
338,2021-03-04 09:00:00+00:00,75.18,70.86,1245.05,19,-0.707107,0.467359,85.628752,156.903984,199.220095,...,238.966779,300.425701,345.177645,375.303556,488.195210,92.00,55.58,33.65,17.34,497.60
339,2021-03-04 09:30:00+00:00,73.19,70.86,1590.00,20,-0.707107,0.467359,89.618646,163.000319,205.983800,...,263.775945,307.808096,356.190088,404.786859,445.870108,92.00,57.95,62.70,17.34,554.50
340,2021-03-04 10:00:00+00:00,74.12,63.72,1415.30,21,-0.866025,0.467359,140.104118,259.247292,313.074122,...,350.917349,469.297198,491.961995,584.346472,624.029818,98.00,53.11,34.10,14.25,739.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,85.584636,147.450968,...,257.545157,312.000836,389.209655,461.471346,565.748439,66.01,91.88,126.00,50.21,1271.20
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,44.062739,80.500728,...,152.946134,213.340381,285.856261,352.111989,494.824570,66.01,90.64,90.64,37.82,1643.15
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-31.030380,1.674412,...,80.555516,138.624007,177.132223,250.514362,333.223882,66.01,95.15,115.99,37.82,1599.25
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-38.988480,19.495393,...,72.199724,128.661011,157.102188,214.652927,345.416021,66.01,91.30,125.00,31.66,1619.70


In [15]:
# Original dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["imbalance_price"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
#save the scaler
pickle.dump(scaler, open("LSTM_imbalance_scaler.pkl", "wb"))

In [16]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [17]:
# y_train = y_train.values
# y_test = y_test.values

In [18]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1                  # Dropout rate
learning_rate = 0.0001         # Learning rate for optimizer
batch_size = 32                # Batch size
num_epochs = 500               # Maximum number of epochs
patience = 15                  # Patience for early stopping
rel_improvement_threshold = 0.0001  # Relative improvement threshold

In [19]:
input_size

15

In [20]:
hidden_size

64

In [21]:
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [22]:
import torch.nn.functional as F  # For MAE loss function

best_val_loss = float('inf')  # Initialize the best validation loss
best_model = None
patience_counter = 0  # For early stopping

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension (for some models)
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Reshape input to add sequence length dimension
            X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

            # Forward pass
            y_pred = model(X_batch)  # (batch_size, output_size)

            # Compute the validation loss using MAE
            val_loss = F.l1_loss(y_pred, y_batch)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Early stopping based on validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model = model.state_dict()  # Save the best model's weights
        patience_counter = 0  # Reset patience counter
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
if best_model is not None:
    model.load_state_dict(best_model)
else:
    print("No improvement was observed during training.")

torch.save(model.state_dict(), 'LSTM_imbalance_price.pth')

# Test the model using the best model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')


Epoch [1/500], Training Loss: 161.8794
Epoch [1/500], Validation Loss: 150.1123
Best model saved with validation loss: 150.1123
Epoch [2/500], Training Loss: 153.2411
Epoch [2/500], Validation Loss: 145.0695
Best model saved with validation loss: 145.0695
Epoch [3/500], Training Loss: 148.6720
Epoch [3/500], Validation Loss: 141.3292
Best model saved with validation loss: 141.3292
Epoch [4/500], Training Loss: 144.6919
Epoch [4/500], Validation Loss: 137.8792
Best model saved with validation loss: 137.8792
Epoch [5/500], Training Loss: 140.9227
Epoch [5/500], Validation Loss: 134.5706
Best model saved with validation loss: 134.5706
Epoch [6/500], Training Loss: 137.2839
Epoch [6/500], Validation Loss: 131.4162
Best model saved with validation loss: 131.4162
Epoch [7/500], Training Loss: 133.7786
Epoch [7/500], Validation Loss: 128.4074
Best model saved with validation loss: 128.4074
Epoch [8/500], Training Loss: 130.3588
Epoch [8/500], Validation Loss: 125.4805
Best model saved with va

In [23]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [24]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new

,timestamp,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-10-03T22:00:00+00:00,2024-10-04,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-03T22:30:00+00:00,2024-10-04,788,584,1148,1320,1007,565,923,797,487
2,2024-10-03T23:00:00+00:00,2024-10-04,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-03T23:30:00+00:00,2024-10-04,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04T00:00:00+00:00,2024-10-04,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
619,2024-10-20T19:30:00+00:00,2024-10-20,457,495,499,512,517,522,524,533,536
620,2024-10-20T20:00:00+00:00,2024-10-20,454,459,493,508,508,517,520,527,532
621,2024-10-20T20:30:00+00:00,2024-10-20,454,464,489,499,504,512,516,529,536
622,2024-10-20T21:00:00+00:00,2024-10-20,351,469,485,500,501,515,522,530,536


In [25]:
from datetime import datetime, timedelta
df_api_new = df_api_new.rename(columns={
    "timestamp": "datetime"
})
df_api_new["datetime"] = pd.to_datetime(df_api_new["datetime"])
min_date = df_api_new.datetime.min() - timedelta(minutes=30)
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
datetimes = pd.date_range(end=min_date, periods=336, freq='30min')
df_half_hourly = pd.DataFrame({"datetime": datetimes})
df_half_hourly["datetime"] = pd.to_datetime(df_half_hourly["datetime"])
df_api_new = pd.merge(df_half_hourly, df_api_new, left_on='datetime', right_on='datetime', how='outer')
df_api_new

,datetime,prediction_date,1,2,3,4,5,6,7,8,9
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
955,2024-10-20 19:30:00+00:00,2024-10-20,457.0,495.0,499.0,512.0,517.0,522.0,524.0,533.0,536.0
956,2024-10-20 20:00:00+00:00,2024-10-20,454.0,459.0,493.0,508.0,508.0,517.0,520.0,527.0,532.0
957,2024-10-20 20:30:00+00:00,2024-10-20,454.0,464.0,489.0,499.0,504.0,512.0,516.0,529.0,536.0
958,2024-10-20 21:00:00+00:00,2024-10-20,351.0,469.0,485.0,500.0,501.0,515.0,522.0,530.0,536.0


In [26]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='datetime', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["datetime"].dt.day/7)
df_api_new_merged

,datetime,prediction_date,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-09-26 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21.21,1485.30,2024-09-26 22:00:00+00:00,2024-09-26,47.0,68.929966,7.20,21.21,0.866025,-0.222521
1,2024-09-26 22:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.46,1364.20,2024-09-26 22:30:00+00:00,2024-09-26,48.0,-28.480000,7.20,3.46,0.866025,-0.222521
2,2024-09-26 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.15,1688.95,2024-09-26 23:00:00+00:00,2024-09-27,1.0,-19.020000,3.87,3.15,0.965926,-0.222521
3,2024-09-26 23:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.68,1661.10,2024-09-26 23:30:00+00:00,2024-09-27,2.0,-18.000000,3.87,-3.68,0.965926,-0.222521
4,2024-09-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.52,1824.60,2024-09-27 00:00:00+00:00,2024-09-27,3.0,56.450000,0.45,-0.52,1.000000,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,2024-10-20 19:30:00+00:00,2024-10-20,457.0,495.0,499.0,512.0,517.0,522.0,524.0,533.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.258819,0.623490
962,2024-10-20 20:00:00+00:00,2024-10-20,454.0,459.0,493.0,508.0,508.0,517.0,520.0,527.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,0.623490
963,2024-10-20 20:30:00+00:00,2024-10-20,454.0,464.0,489.0,499.0,504.0,512.0,516.0,529.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,0.623490
964,2024-10-20 21:00:00+00:00,2024-10-20,351.0,469.0,485.0,500.0,501.0,515.0,522.0,530.0,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,0.623490


In [27]:
df_api_new_merged1 = df_api_new_merged[["datetime","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(336)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(192)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,1330.0,867.0,326.0,1402.0,1357.0,61.00,12.85,-1.4600,7.20,1457.25
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,1007.0,565.0,923.0,797.0,487.0,50.00,-3.95,-1.4600,7.20,1367.60
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,740.0,1017.0,1533.0,597.0,991.0,93.50,-2.74,5.0000,3.87,1473.20
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,1054.0,1572.0,744.0,1497.0,463.0,93.50,3.85,5.0000,3.87,1799.05
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,1505.0,1093.0,814.0,487.0,1340.0,93.00,0.03,-2.4304,0.45,1689.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,0.258819,0.623490,850.0,921.0,980.0,...,1060.0,1081.0,1104.0,1123.0,1142.0,105.00,84.79,66.0000,89.55,2107.60
722,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,0.500000,0.623490,897.0,964.0,1010.0,...,1074.0,1097.0,1111.0,1129.0,1151.0,104.90,85.72,66.6000,87.15,2231.60
723,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,0.500000,0.623490,910.0,984.0,1040.0,...,1101.0,1120.0,1122.0,1139.0,1154.0,64.00,88.05,69.0000,87.15,1923.85
724,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,0.707107,0.623490,938.0,1034.0,1069.0,...,1124.0,1135.0,1139.0,1152.0,1164.0,33.77,85.14,85.1400,81.06,1929.50


In [28]:
X_test = df_api_new_merged1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
X_test_tensor.shape

torch.Size([388, 15])

In [29]:
X_test_tensor = X_test_tensor.unsqueeze(1)  # Adds a sequence length dimension


In [30]:


model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["imbalance_price_predictions"] = final_test_outputs
df_api_new_merged1

,datetime,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag96h,imbalance_price_lag96h,day_ahead_price_lag1week,volume_lag96h,imbalance_price_predictions
336,2024-10-03 22:00:00+00:00,71.32,78.47,1322.45,47.0,0.866025,-0.900969,1355.0,487.0,963.0,...,867.0,326.0,1402.0,1357.0,61.00,12.85,-1.4600,7.20,1457.25,66.665184
337,2024-10-03 22:30:00+00:00,71.42,78.47,1035.65,48.0,0.866025,-0.900969,788.0,584.0,1148.0,...,565.0,923.0,797.0,487.0,50.00,-3.95,-1.4600,7.20,1367.60,45.748684
338,2024-10-03 23:00:00+00:00,66.94,71.36,1378.55,1.0,0.965926,-0.900969,571.0,748.0,1001.0,...,1017.0,1533.0,597.0,991.0,93.50,-2.74,5.0000,3.87,1473.20,58.155209
339,2024-10-03 23:30:00+00:00,60.03,71.36,1539.90,2.0,0.965926,-0.900969,1476.0,1194.0,1512.0,...,1572.0,744.0,1497.0,463.0,93.50,3.85,5.0000,3.87,1799.05,61.912571
340,2024-10-04 00:00:00+00:00,65.21,72.80,1818.40,3.0,1.000000,-0.900969,1352.0,1390.0,912.0,...,1093.0,814.0,487.0,1340.0,93.00,0.03,-2.4304,0.45,1689.75,58.557213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,0.258819,0.623490,850.0,921.0,980.0,...,1081.0,1104.0,1123.0,1142.0,105.00,84.79,66.0000,89.55,2107.60,99.317078
722,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,0.500000,0.623490,897.0,964.0,1010.0,...,1097.0,1111.0,1129.0,1151.0,104.90,85.72,66.6000,87.15,2231.60,98.818047
723,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,0.500000,0.623490,910.0,984.0,1040.0,...,1120.0,1122.0,1139.0,1154.0,64.00,88.05,69.0000,87.15,1923.85,100.117615
724,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,0.707107,0.623490,938.0,1034.0,1069.0,...,1135.0,1139.0,1152.0,1164.0,33.77,85.14,85.1400,81.06,1929.50,96.228027


In [31]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["imbalance_price"], mode='lines', name='Imbalance Price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["datetime"], y=df_api_new_merged1["imbalance_price_predictions"], mode='lines', name='Imbalance Price Predictions'))
fig.update_layout(title='Imbalance Price and Predictions', xaxis_title='Timestamp', yaxis_title='Imbalance Price')
fig.show()

In [32]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged1["imbalance_price"] - df_api_new_merged1["imbalance_price_predictions"]))
print(f"MAE for imbalance price predictions: {mae:.4f}")

MAE for imbalance price predictions: 45.1196


In [35]:
df_api_new_merged1.drop(columns=["datetime"]).corr()["imbalance_price"]

market_price                   0.056896
day_ahead_price                0.590410
volume                        -0.210950
settlement_period              0.243120
cos_hour                      -0.309917
cos_day                        0.178976
1                              0.033391
2                              0.041091
3                              0.047547
4                              0.037825
5                              0.045943
6                              0.055354
7                              0.046126
8                              0.065398
9                              0.070242
imbalance_price                1.000000
market_price_lag96h            0.192127
imbalance_price_lag96h         0.044277
day_ahead_price_lag1week       0.068846
volume_lag96h                  0.230833
imbalance_price_predictions   -0.098272
Name: imbalance_price, dtype: float64

In [33]:
#predict for the whole dataset
X = df_bbidding1[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X = scaler.transform(X)
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor = X_tensor.unsqueeze(1)  # Adds a sequence length

model.eval()
with torch.no_grad():
    final_outputs = model(X_tensor)
    final_outputs = final_outputs.numpy().flatten()

df_bbidding1["imbalance_price_predictions"] = final_outputs

In [34]:
df_bbidding1.to_csv('bidding_training_predictions_imbalance_price.csv', index=False)